In [1]:
import tensorflow as tf
from tensorflow import keras

import externalTensor as exT
import numpy as np
from sklearn.model_selection import KFold
from boxScore import boxScore
import lime
from lime import lime_tabular
import pandas as pd

2022-10-05 13:40:36.690069: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-05 13:40:36.690088: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


# Preparation of Data

In [2]:
years="2021-22"
# stats="traditional"
stats='advance'
box_score=boxScore(years,stats)

x_train, x_test, y_train, y_test=box_score.separation()
y_train=np.array(y_train)


#  Load Model

In [ ]:
# bestModelArray[2].save("saved_model_"+stats+"/"+"bestModel")
model = tf.keras.models.load_model('saved_model_'+stats+'/bestModel') 



# Control matchid

In [7]:


MatchId=[]
nameFile="/MatchId"+years+".txt"

with open(r"/home/sergio/UniSergio/BasketScript/WaB/outcomebasketmatch/BoxScoresFile"+nameFile,'r') as f:
        fr=f.readlines()  
        for line in fr:
            line=line.replace('\n', '')
           
            # We take the first 650 match 
            if(int(line[-3:]) <=650 and int(line[-4])==0):
                MatchId.append(line)





In [8]:
model = tf.keras.models.load_model('saved_model_'+stats+'/bestModel') 
X=np.asarray(x_test)
Y=np.asarray(y_test)
model.evaluate(X,Y)
pred=model.predict(X)

indices = [i for i,v in enumerate(pred) if (np.rint(pred[i])!=y_test[i]).all()]
subset_of_wrongly_predicted = [x_test.iloc[i].name for i in indices ]


2/2 [==============================] - 0s 1ms/step - loss: 0.9518 - accuracy: 0.6000


20

In [11]:
from nba_api.stats.endpoints import boxscoretraditionalv2 
x=MatchId[subset_of_wrongly_predicted[2]]
df=boxscoretraditionalv2.BoxScoreTraditionalV2(game_id=x).get_data_frames()[1]
df

,GAME_ID,TEAM_ID,TEAM_NAME,TEAM_ABBREVIATION,TEAM_CITY,MIN,FGM,FGA,FG_PCT,FG3M,...,OREB,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS
0,0022100407,1610612745,Rockets,HOU,Houston,240:00,46,96,0.479,19,...,12,33,45,27,8,5,11,24,132,6.0
1,0022100407,1610612737,Hawks,ATL,Atlanta,240:00,42,85,0.494,15,...,8,32,40,20,9,6,12,26,126,-6.0


# LIME 

In [ ]:


from locale import atof

model = tf.keras.models.load_model('saved_model_'+stats+'/bestModel') 
explainer = lime_tabular.LimeTabularExplainer(
                        training_data=np.array(x_train),
                        feature_names=x_train.columns,
                        class_names=['win first team', 'win second team'],
                        mode='classification'
                    )
answer=[]

list_index=[0,1,2,6]
# list_index=[6]
big_one=[]
for i in list_index:
    r=np.asarray(x_test)
    exp = explainer.explain_instance(
                data_row=x_test.iloc[i], 
                predict_fn=model.predict
            )
    prediction=model.predict(r[i][None,...])
    
    boolean_vector=np.rint(prediction)==y_test[i]
    value_boolean_dictionary=boolean_vector.all()



    res=exp.as_list()
    tmp=np.array(res)
    value_predestination=tmp[:,1].astype(float)
    total_sum_abs=np.sum( np.abs(value_predestination))
    tec=(value_predestination)/total_sum_abs
    
    

    string_predestination=tmp[:,0]
    
    answer=[ [x,z] for x,z in zip(string_predestination,tec)]
    

    dict=[]
    for count,ele in enumerate(string_predestination[0:4]):
        ecc=ele.split()
        range_div=[]
        for el in ecc:
            if el[0].isalpha():
                word=el
            else:
                if el!='<' and el!='<=' and el!='>=' and el!='>':
                    range_div.append(atof(el))
                else:
                    operator=el
        tmp=x_test.iloc[i][word]
        
        if(len(range_div)>1):

            dict.append([(range_div[0],range_div[1]),tmp,word,tec[count],value_boolean_dictionary,i])
        else:
            dict.append([ (range_div[0],operator),tmp,word,tec[count],value_boolean_dictionary,i])

    big_one.append(dict)


In [ ]:
big_one=np.array(big_one)
big_one=big_one.reshape(16,6)



result=pd.DataFrame(big_one)
result.columns=["Range","Effective value","Stats", "Percentage of decision", "Correct prediction","id"]
result
# result[result["Stats"].str.contains("OFF_RATING")]

In [ ]:
conclusion=[]
absvalue=[]
counts={}
oblio=0
for list in answer:
    for el in list[0:4]:
        
        stat=[word for word in el[0].split() if word[0].isalpha()] 
        conclusion.append([stat[0],abs(el[1])])
        if stat[0]  not in counts: 
            counts[stat[0]]=abs(el[1])
        else:
            
            counts[stat[0]]=counts.get(stat[0])+abs(el[1])

    oblio=1


sortdict = dict(sorted(counts.items(), key=lambda x:x[1],reverse=True))


In [ ]:
# sortdict